 Шаг 1.
 Залил данные на YT, посчитал для каждого фильма и каждого пользователя перцентили оценок. 
 Приджойнил эти данные к таблицам train и test, которые использую ниже

In [1]:
import pandas as pd
import numpy as np
from functools import partial
import json
from collections import defaultdict

In [2]:
# train = pd.DataFrame()

columns=['id', 'rating'] + ["mp0{}".format(i) for i in range(1, 10)] + ["up0{}".format(i) for i in range(1, 10)]
big_dict = defaultdict(list)
with open('big_train', 'r') as f:
    for line in f:
        l = json.loads(line)
        for c in columns:
            big_dict[c].append(l[c])

train = pd.DataFrame(columns=columns)
for c in columns:
    train[c] = big_dict[c]
    
del big_dict

train.head()

,id,rating,mp01,mp02,mp03,mp04,mp05,mp06,mp07,mp08,mp09,up01,up02,up03,up04,up05,up06,up07,up08,up09
0,11244_8081311342647038241,4.0,2.5,3.0,3.5,3.5,4.000000,4.000000,4.000000,4.5,5.0,1.65,2.55,3.5,3.5,4.0,4.000,4.5,5.0,5.0
1,1273_7014923320312956287,5.0,3.0,3.0,3.5,4.0,4.000000,4.000000,4.004091,5.0,5.0,3.00,4.00,4.0,4.0,4.0,4.000,4.0,4.0,5.0
2,3027_16900488111374893251,1.0,1.0,2.0,2.0,2.5,3.000000,3.000000,3.000000,3.5,4.0,1.00,2.50,3.5,4.0,4.0,4.350,4.5,4.5,5.0
3,3088_11958362235749243305,4.0,2.0,3.0,3.0,3.5,3.725975,4.000000,4.000000,4.0,4.5,2.00,2.50,3.0,3.0,3.5,3.500,3.5,4.0,4.0
4,3816_15802792521601871812,4.0,2.0,2.5,3.0,3.0,3.500000,3.984711,4.000000,4.0,4.5,2.50,3.00,3.0,3.5,3.5,3.671,4.0,4.0,4.5


In [3]:
# train = pd.DataFrame()

columns=['id'] + ["mp0{}".format(i) for i in range(1, 10)] + ["up0{}".format(i) for i in range(1, 10)]
big_dict = defaultdict(list)
with open('big_test', 'r') as f:
    for line in f:
        l = json.loads(line)
        for c in columns:
            big_dict[c].append(l[c])

test = pd.DataFrame(columns=columns)
for c in columns:
    test[c] = big_dict[c]
    
del big_dict

test.head()

,id,mp01,mp02,mp03,mp04,mp05,mp06,mp07,mp08,mp09,up01,up02,up03,up04,up05,up06,up07,up08,up09
0,1565_1180382074245455203,3.0,3.5,4.0,4.0,4.000000,4.083290,4.502332,5.000000,5.0,1.5,2.5,2.50,3.0,3.5,4.0,4.0,4.5,5.0
1,603_8339124594073376442,3.0,3.5,4.0,4.0,4.000000,4.500000,5.000000,5.000000,5.0,3.0,4.0,4.00,4.0,4.0,4.5,5.0,5.0,5.0
2,587_12165890619449965271,2.5,3.0,3.0,3.0,3.003184,3.833483,4.000000,4.000000,4.5,1.0,3.0,3.00,3.0,4.0,4.0,4.0,4.0,4.0
3,1538_11621974557331621128,2.0,3.0,3.0,3.0,3.000000,3.500000,4.000000,4.000000,4.0,1.0,2.0,2.65,3.5,3.5,4.0,4.5,5.0,5.0
4,160_12938098507280739324,3.0,3.0,3.0,3.5,4.000000,4.000000,4.000000,4.024778,5.0,2.0,3.0,3.00,3.0,4.0,4.0,4.0,5.0,5.0


In [4]:
features_columns = ["mp0{}".format(i) for i in range(1, 10)] + ["up0{}".format(i) for i in range(1, 10)]

In [5]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
est = RandomForestRegressor(n_estimators=300)
est.fit(train[features_columns], train.rating)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [35]:
res = est.predict(test[features_columns])

In [36]:
resd = dict()

for k, v in zip(test.id, res):
    resd[k] = v

In [38]:
cnt = 0 
with open('test.csv', 'r') as f:
    with open('answ2', 'a') as out:
        for line in f:
            m,u = line.strip().split(',')
            if cnt == 0:
                out.write(line)
                cnt += 1
                continue
            out.write("{},{}\n".format(cnt, resd[m + "_" + u]))
            cnt += 1